# PyPSA_DSM_risk: Germany

**This model has 585 nodes for the whole Germany which can be simulated for 8760 timesteps for the year 2023**

**Make sure you have following packages installed before importing them**

### 1)Import Packages

In [43]:
import pypsa
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from urllib.request import urlretrieve
import cartopy.crs as ccrs
from pypsa.descriptors import get_switchable_as_dense as as_dense
plt.style.use("bmh")
import re

### 2)Create a network and set Snapshots

In [44]:
# Create a new PyPSA network
network = pypsa.Network()
solver='highs' #Use the solver that you have installed

In [45]:
# Define snapshots
snapshots=pd.date_range(start="2023-01-01 00:00", end="2023-12-31 23:00", freq="h")
network.set_snapshots(snapshots)

### 3)Add Buses

In [46]:
# Read excel file which contains Non renewable generators data
def add_buses(filename,header):
    try:
        bus_data = pd.read_csv(filename,header=header)
    except pd.errors.EmptyDataError:
        print("The CSV file is empty.")
        return
        
    network.madd(
        "Bus",
        names=list(bus_data.Bus),
        v_nom=list(bus_data.v_nom),
        x=list(bus_data.x),
        y=list(bus_data.y),
        carrier=list(bus_data.carrier),
        control=list(bus_data.control)
        )
    return network.buses.head(3)

add_buses('buses.csv',0)

,v_nom,x,y,carrier,control,type,unit,v_mag_pu_set,v_mag_pu_min,v_mag_pu_max,generator,sub_network
Bus,,,,,,,,,,,,
1,220.0,9.522576,52.360409,AC,Slack,,,1.0,0.0,inf,,
2,380.0,9.113210,52.543853,AC,PQ,,,1.0,0.0,inf,,
3,380.0,9.389745,52.026313,AC,PQ,,,1.0,0.0,inf,,


### 4)Add generators

##### A) Add dispatchables generators

In [47]:
generators_disp = pd.read_csv('generators_disp.csv')

network.madd(
    "Generator",
    generators_disp.name,
    bus=list(generators_disp.bus),
    carrier=list(generators_disp.carrier),
    p_max_pu=1,
    p_min_pu=0,
    p_nom=list(generators_disp.p_nom),
    marginal_cost=list(generators_disp.marginal_cost),
    sign=1
)
network.generators

,bus,carrier,p_max_pu,p_min_pu,p_nom,marginal_cost,sign,control,type,p_nom_mod,...,min_up_time,min_down_time,up_time_before,down_time_before,ramp_limit_up,ramp_limit_down,ramp_limit_start_up,ramp_limit_shut_down,weight,p_nom_opt
Generator,,,,,,,,,,,,,,,,,,,,,
Unit_1,122,Natural gas,1.0,0.0,21.5,46.1,1.0,PQ,,0.0,...,0,0,1,0,NaN,NaN,1.0,1.0,1.0,0.0
Unit_2,394,Natural gas,1.0,0.0,78.3,50.5,1.0,PQ,,0.0,...,0,0,1,0,NaN,NaN,1.0,1.0,1.0,0.0
Unit_3,12,Natural gas,1.0,0.0,148.4,60.7,1.0,PQ,,0.0,...,0,0,1,0,NaN,NaN,1.0,1.0,1.0,0.0
Unit_4,226,Natural gas,1.0,0.0,11.0,61.5,1.0,PQ,,0.0,...,0,0,1,0,NaN,NaN,1.0,1.0,1.0,0.0
Unit_5,378,Natural gas,1.0,0.0,248.0,66.2,1.0,PQ,,0.0,...,0,0,1,0,NaN,NaN,1.0,1.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Unit_427,34,Oil,1.0,0.0,56.0,190.6,1.0,PQ,,0.0,...,0,0,1,0,NaN,NaN,1.0,1.0,1.0,0.0
Unit_428,61,Hard coal,1.0,0.0,350.0,52.0,1.0,PQ,,0.0,...,0,0,1,0,NaN,NaN,1.0,1.0,1.0,0.0
Unit_429,299,Hard coal,1.0,0.0,690.0,49.1,1.0,PQ,,0.0,...,0,0,1,0,NaN,NaN,1.0,1.0,1.0,0.0


#### B) Add renewable generators

In [48]:
generators_renew = pd.read_csv('generators_renew.csv')
gen_p_max_pu = pd.read_csv('gen_p_max_pu.csv', index_col=0, parse_dates=True)

network.madd(
    "Generator",
    generators_renew.name,
    bus=list(generators_renew.bus),
    carrier=list(generators_renew.carrier),
    p_nom=list(generators_renew.p_nom),
    p_max_pu=gen_p_max_pu,
    p_min_pu=0,
    marginal_cost=list(generators_renew.marginal_cost),
    sign=1
)
network.generators

       '101_Wind offshore1950', '101_Wind onshore1934', '102_Biomass1789',
       '111_Biomass195', '111_Biomass196', '119_Hydro657', '119_Hydro658',
       ...
       '73_Biomass1611', '77_Hydro356', '77_Hydro357', '78_Biomass1422',
       '78_Biomass1439', '7_Biomass1244', '81_Biomass69', '87_Biomass162',
       '87_Biomass163', '87_Wind offshore1959'],
      dtype='object', name='Generator', length=552) for attribute p_max_pu of Generator are not in main components dataframe generators


,bus,carrier,p_max_pu,p_min_pu,p_nom,marginal_cost,sign,control,type,p_nom_mod,...,min_up_time,min_down_time,up_time_before,down_time_before,ramp_limit_up,ramp_limit_down,ramp_limit_start_up,ramp_limit_shut_down,weight,p_nom_opt
Generator,,,,,,,,,,,,,,,,,,,,,
Unit_1,122,Natural gas,1.0,0.0,21.5,46.1,1.0,PQ,,0.0,...,0,0,1,0,NaN,NaN,1.0,1.0,1.0,0.0
Unit_2,394,Natural gas,1.0,0.0,78.3,50.5,1.0,PQ,,0.0,...,0,0,1,0,NaN,NaN,1.0,1.0,1.0,0.0
Unit_3,12,Natural gas,1.0,0.0,148.4,60.7,1.0,PQ,,0.0,...,0,0,1,0,NaN,NaN,1.0,1.0,1.0,0.0
Unit_4,226,Natural gas,1.0,0.0,11.0,61.5,1.0,PQ,,0.0,...,0,0,1,0,NaN,NaN,1.0,1.0,1.0,0.0
Unit_5,378,Natural gas,1.0,0.0,248.0,66.2,1.0,PQ,,0.0,...,0,0,1,0,NaN,NaN,1.0,1.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Unit_1353,36,wind offshore,1.0,0.0,718.7,0.0,1.0,PQ,,0.0,...,0,0,1,0,NaN,NaN,1.0,1.0,1.0,0.0
Unit_1354,18,wind offshore,1.0,0.0,744.1,0.0,1.0,PQ,,0.0,...,0,0,1,0,NaN,NaN,1.0,1.0,1.0,0.0
Unit_1355,271,wind offshore,1.0,0.0,253.7,0.0,1.0,PQ,,0.0,...,0,0,1,0,NaN,NaN,1.0,1.0,1.0,0.0


### 5)Add load

In [49]:
# Read excel file which contains Non renewable generators data
def add_consumers(filename):
    try:
        loads = pd.read_csv(filename)
        load_p_set = pd.read_csv('load_p_set.csv',index_col=0, parse_dates=True)
    except pd.errors.EmptyDataError:
        print("The CSV file is empty.")
        return
    
    network.madd(
        "Load",
        loads.load,
        bus=list(loads.bus),
        p_set=load_p_set,
        sign=-1
    )
    return network.loads_t.p_set

add_consumers('loads.csv')

Load,1,3,4,6,7,8,9,11,14,16,...,382_220kV,384_220kV,385_220kV,391_220kV,403_220kV,404_220kV,413_220kV,421_220kV,450_220kV,458_220kV
snapshot,,,,,,,,,,,,,,,,,,,,,
2023-01-01 00:00:00,184.243515,32.292932,53.106798,155.943574,117.522875,98.334745,66.502251,58.267308,139.354027,237.663209,...,160.623436,177.070594,169.061074,61.678094,12.840461,0.073783,46.456855,53.284305,30.571982,53.076723
2023-01-01 01:00:00,179.694624,31.495634,51.795615,152.093396,114.621287,95.906904,64.860340,56.828714,135.913438,231.795409,...,156.657715,172.698799,164.887031,60.155289,12.523436,0.071961,45.309856,51.968739,29.817173,51.766282
2023-01-01 02:00:00,175.087528,30.688134,50.467655,148.193954,111.682573,93.447997,63.197420,55.371713,132.428825,225.852529,...,152.641251,168.271066,160.659580,58.612999,12.202354,0.070116,44.148180,50.636340,29.052707,50.439074
2023-01-01 03:00:00,173.560784,30.420537,50.027582,146.901719,110.708713,92.633141,62.646346,54.888878,131.274060,223.883120,...,151.310236,166.803761,159.258646,58.101901,12.095951,0.069504,43.763213,50.194797,28.799370,49.999251
2023-01-01 04:00:00,174.348781,30.558652,50.254716,147.568679,111.211350,93.053712,62.930771,55.138083,131.870068,224.899589,...,151.997211,167.561080,159.981708,58.365693,12.150869,0.069820,43.961906,50.422690,28.930125,50.226256
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-01-10 19:00:00,239.260022,41.935846,68.964890,202.509505,152.616095,127.698244,86.360327,75.666368,180.966194,308.631241,...,208.586810,229.945213,219.543990,80.095639,16.674720,0.095815,60.329223,69.195402,39.701007,68.925834
2023-01-10 20:00:00,228.657883,40.077576,65.908904,193.535862,145.853339,122.039653,82.533510,72.313424,172.947183,294.955110,...,199.343868,219.755833,209.815511,76.546425,15.935827,0.091569,57.655902,66.129201,37.941768,65.871578
2023-01-10 21:00:00,225.434259,39.512562,64.979719,190.807389,143.797095,120.319136,81.369951,71.293948,170.508970,290.796827,...,196.533514,216.657711,206.857528,75.467272,15.711163,0.090278,56.843068,65.196910,37.406864,64.942919


### 6)Add transformers

In [50]:
# Read excel file which contains Non renewable generators data
def add_transformers(filename):
    try:
        transformers = pd.read_csv(filename)
    except pd.errors.EmptyDataError:
        print("The CSV file is empty.")
        return
    
    network.madd(
        "Transformer",
        transformers.Transformer,
        bus0=list(transformers.bus0),
        bus1=list(transformers.bus1),
        s_nom=list(transformers.s_nom),
        s_nom_extendable=list(transformers.s_nom_extendable),
        x=list(transformers.x),
        r=0.0001
    )
    return network.transformers

add_transformers('transformers.csv')

,bus0,bus1,s_nom,s_nom_extendable,x,r,type,model,g,b,...,v_ang_min,v_ang_max,sub_network,x_pu,r_pu,g_pu,b_pu,x_pu_eff,r_pu_eff,s_nom_opt
Transformer,,,,,,,,,,,,,,,,,,,,,
2,2,2_220kV,2000.0,False,0.1,0.0001,,t,0.0,0.0,...,-inf,inf,,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,5,5_220kV,2000.0,False,0.1,0.0001,,t,0.0,0.0,...,-inf,inf,,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10,10,10_220kV,2000.0,False,0.1,0.0001,,t,0.0,0.0,...,-inf,inf,,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12,12,12_220kV,2000.0,False,0.1,0.0001,,t,0.0,0.0,...,-inf,inf,,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13,13,13_220kV,2000.0,False,0.1,0.0001,,t,0.0,0.0,...,-inf,inf,,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
404,404,404_220kV,2000.0,False,0.1,0.0001,,t,0.0,0.0,...,-inf,inf,,0.0,0.0,0.0,0.0,0.0,0.0,0.0
413,413,413_220kV,2000.0,False,0.1,0.0001,,t,0.0,0.0,...,-inf,inf,,0.0,0.0,0.0,0.0,0.0,0.0,0.0
421,421,421_220kV,2000.0,False,0.1,0.0001,,t,0.0,0.0,...,-inf,inf,,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### 7)Add Transmission lines

In [51]:
def add_lines(filename, header):
    try:
        lines = pd.read_csv(filename, header=header)
    except pd.errors.EmptyDataError:
        print("The CSV file is empty.")
        return
        
    network.madd(
        "Line",
        lines.Line,
        bus0=list(lines.bus0),
        bus1=list(lines.bus1),
        s_nom=100000,
        s_nom_extendable=True,
        x=1,
        s_max_pu=1,
        capital_cost=0,
        r=0.000001
    )
    return network.lines.head(3)

add_lines('lines.csv',0)

,bus0,bus1,s_nom,s_nom_extendable,x,s_max_pu,capital_cost,r,type,g,...,v_ang_min,v_ang_max,sub_network,x_pu,r_pu,g_pu,b_pu,x_pu_eff,r_pu_eff,s_nom_opt
Line,,,,,,,,,,,,,,,,,,,,,
1,1,2_220kV,100000.0,True,1.0,1.0,0.0,0.000001,,0.0,...,-inf,inf,,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,4,100000.0,True,1.0,1.0,0.0,0.000001,,0.0,...,-inf,inf,,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,5_220kV,6,100000.0,True,1.0,1.0,0.0,0.000001,,0.0,...,-inf,inf,,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [52]:
# Check what components are added to your PyPSA network
display(network)

PyPSA Network
Components:
 - Bus: 585
 - Generator: 1336
 - Line: 852
 - Load: 489
 - Transformer: 96
Snapshots: 240

In [ ]:
#Solve the PyPSA Network
network.optimize(solver_name='highs')

### 8)Outputs

#### A) Overall Statistics

In [ ]:
stats=network.statistics()
stats.to_csv("./outputs/statistics.csv")
stats

#### B) Marginal Price

In [ ]:
marginal_prices= network.buses_t.marginal_price
marginal_prices.to_csv('./outputs/marginal_price.csv')

#### C) Hourly generations from each powerplants

In [ ]:
generation=network.generators_t.p
generation.to_csv("./outputs/gen_p_set.csv")

### 9)Visualizations

#### A)Grid loading

In [ ]:
now = network.snapshots[11]
loading = network.lines_t.p0.loc[now] / network.lines.s_nom
loading.describe()
fig, ax = plt.subplots(subplot_kw={"projection": ccrs.EqualEarth()}, figsize=(6, 6))
network.plot(
    ax=ax,
    line_colors=abs(loading),
    line_cmap=plt.cm.jet,
    title="Line loading",
    bus_sizes=1e-3,
    bus_alpha=0.7,
)
fig.tight_layout()